In [21]:
import earthaccess
import xarray as xr
from xarray.backends.api import open_datatree
import matplotlib.pyplot as plt
import numpy as np

In [22]:
auth = earthaccess.login(persist=True)

In [ ]:
results = earthaccess.search_datasets(
    keyword="L2 ocean color",
    instrument="olci",
)
set((i.summary()["short-name"] for i in results))

set()

In [25]:
tspan = ("2020-10-15", "2020-10-23")
bbox = (-89.4474, 30.064, -88.337, 30.064)
cc = (0, 50)
results = earthaccess.search_data(
    short_name="OLCIS3A_L2_EFR_OC",
    temporal=tspan,
    bounding_box=bbox,
    cloud_cover=cc,
)

In [27]:
paths = earthaccess.download(results, "data")

QUEUEING TASKS | :   0%|          | 0/4 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/4 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
paths[0]

'data\\S3A_OLCI_EFRNT.20201015T155218.L2.OC.nc'

In [62]:
from affine import Affine
prod = xr.open_dataset(paths[0])
obs = xr.open_dataset(paths[0], group="geophysical_data")
nav = xr.open_dataset(paths[0], group="navigation_data")
nav = (
    nav
    .set_coords(("longitude", "latitude"))
    .rename({"pixel_control_points": "pixels_per_line"})
)
dataset = xr.merge((prod, obs, nav.coords))
# set latitude and longitude as coordinates
dataset = dataset.set_coords(("latitude", "longitude"))
# slice (-89.4474, 30.064, -88.337, 30.064)
# dataset = dataset.sel(
#     latitude=slice(30.064, 30.064),
#     longitude=slice(-89.4474, -88.337),
# )
dataset

<xarray.Dataset> Size: 1GB
Dimensions:    (number_of_lines: 4091, pixels_per_line: 4865)
Coordinates:
    longitude  (number_of_lines, pixels_per_line) float32 80MB ...
    latitude   (number_of_lines, pixels_per_line) float32 80MB ...
Dimensions without coordinates: number_of_lines, pixels_per_line
Data variables: (12/16)
    aot_865    (number_of_lines, pixels_per_line) float32 80MB ...
    angstrom   (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_400    (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_412    (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_443    (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_490    (number_of_lines, pixels_per_line) float32 80MB ...
    ...         ...
    Rrs_674    (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_681    (number_of_lines, pixels_per_line) float32 80MB ...
    Rrs_709    (number_of_lines, pixels_per_line) float32 80MB ...
    chlor_a    (number_of_lines, pixels_per_line) float32 80MB ...
    Kd_490     (number_of_lines, pixels_per_line) float32 80MB ...
    l2_flags   (number_of_lines, pixels_per_line) int32 80MB ...
Attributes: (12/46)
    title:                          OLCIS3A Level-2 Data
    product_name:                   S3A_OLCI_EFRNT.20201015T155218.L2.OC.nc
    processing_version:             R2022.0
    orbit_number:                   24282
    history:                        l2gen par=/data9/sdpsoper/vdc/vpu20/workb...
    instrument:                     OLCI
    ...                             ...
    geospatial_lon_max:             -79.75462
    geospatial_lon_min:             -94.42871
    startDirection:                 Descending
    endDirection:                   Descending
    day_night_flag:                 Day
    earth_sun_distance_correction:  1.0060665607452393

In [69]:
import datatree

ModuleNotFoundError: No module named 'datatree'

In [42]:
array = dataset["Rrs_709"]
array.attrs.update(
    {
        "units": f'({dataset["Rrs_709"].attrs["units"]})',
    }
)

array
# save to geotiff
array
# array.plot(
#     x="longitude", y="latitude", aspect=2, size=4, cmap="jet", robust=True
# )

<xarray.DataArray 'Rrs_709' (number_of_lines: 4091, pixels_per_line: 4865)> Size: 80MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
    longitude  (number_of_lines, pixels_per_line) float32 80MB ...
    latitude   (number_of_lines, pixels_per_line) float32 80MB ...
Dimensions without coordinates: number_of_lines, pixels_per_line
Attributes:
    long_name:         Remote sensing reflectance at 709 nm
    units:             ((((((((((((sr^-1))))))))))))
    standard_name:     surface_ratio_of_upwelling_radiance_emerging_from_sea_...
    valid_min:         -30000
    valid_max:         25000
    solar_irradiance:  1403.1101

In [ ]:
results = earthaccess.search_datasets(instrument="olci")
for item in results:
    summary = item.summary()
    print(summary["short-name"])
    #OLCIS3A_L2_EFR_OC


S3A_OL_1_EFR
MERGED_S3_OLCI_L3b_CYANTC
MERGED_S3_OLCI_L3b_ILW
MERGED_S3_OLCI_L3m_CYAN
MERGED_S3_OLCI_L3m_CYANTC
MERGED_S3_OLCI_L3m_ILW
OLCIS3A_L1_EFR
OLCIS3A_L1_ERR
OLCIS3A_L2_EFR_IOP
OLCIS3A_L2_EFR_IOP_NRT
OLCIS3A_L2_EFR_OC
OLCIS3A_L2_EFR_OC_NRT
OLCIS3A_L2_ERR_IOP
OLCIS3A_L2_ERR_IOP_NRT
OLCIS3A_L2_ERR_OC
OLCIS3A_L2_ERR_OC_NRT
OLCIS3A_L2_ILW
OLCIS3A_L3b_ERR_CHL
OLCIS3A_L3b_ERR_CHL_NRT
OLCIS3A_L3b_ERR_IOP
OLCIS3A_L3b_ERR_IOP_NRT
OLCIS3A_L3b_ERR_KD
OLCIS3A_L3b_ERR_KD_NRT
OLCIS3A_L3b_ERR_RRS
OLCIS3A_L3b_ERR_RRS_NRT
OLCIS3A_L3b_ILW
OLCIS3A_L3m_ERR_CHL
OLCIS3A_L3m_ERR_CHL_NRT
OLCIS3A_L3m_ERR_IOP
OLCIS3A_L3m_ERR_IOP_NRT
OLCIS3A_L3m_ERR_KD
OLCIS3A_L3m_ERR_KD_NRT
OLCIS3A_L3m_ERR_RRS
OLCIS3A_L3m_ERR_RRS_NRT
OLCIS3A_L3m_ILW
OLCIS3B_L1_EFR
OLCIS3B_L1_ERR
OLCIS3B_L2_EFR_IOP
OLCIS3B_L2_EFR_IOP_NRT
OLCIS3B_L2_EFR_OC
OLCIS3B_L2_EFR_OC_NRT
OLCIS3B_L2_ERR_IOP
OLCIS3B_L2_ERR_IOP_NRT
OLCIS3B_L2_ERR_OC
OLCIS3B_L2_ERR_OC_NRT
OLCIS3B_L2_ILW
OLCIS3B_L3b_ERR_CHL
OLCIS3B_L3b_ERR_CHL_NRT
OLCIS3B_L3b_ERR_IOP
OLC

In [ ]:
results = earthaccess.search_data(
    short_name="OLCIS3A_L2_EFR_OC",
    count=1,
)
tspan = ("2024-01-01", "2024-10-31")
bbox = (-89.4474, 30.064, -88.337, 30.064)
clouds = (0, 50)
results = earthaccess.search_data(
    short_name="OLCIS3A_L2_EFR_OC",
    temporal=tspan,
    bounding_box=bbox,
    cloud_cover=clouds,
)
paths = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/86 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/86 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/86 [00:00<?, ?it/s]

In [20]:
dataset = xr.open_dataset(paths[0], engine='netcdf4')


ValueError: can only read bytes or file-like objects with engine='scipy' or 'h5netcdf'

In [13]:
dataset = xr.open_dataset(paths[0])
dataset

ValueError: The HTTP server doesn't appear to support range requests. Only reading this file from the beginning is supported. Open with block_size=0 for a streaming file interface.